In [ ]:
#Suppress warnings thrown by different packages
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import datetime as dt

# Set Pandas Options to Display all rows & columns when displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set float places & displayed decimal digits
pd.options.display.float_format = '{:13,.2f}'.format

import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pty
import re

In [ ]:
# Read the Dataset
df = pd.read_csv(r"C:\Users\Ashis\Desktop\ML Project\Project 1\books.csv")
print('Dataframe Shape :', df.shape,'\n')

# No null values. But there's clearly a problem with 1 of the column names.
print(df.isna().sum())

# Delete the whitespace in column names
df.columns = df.columns.str.strip()
print('Dataframe columns :', df.columns,'\n')

In [ ]:
# checking for duplicates. No duplicates found
df[df.duplicated(keep=False)]

In [ ]:
# Replace unwanted characters & whitespaces in author names
df.authors = [re.sub(r"\s+", " ", s) for s in df.authors]
df.title  = [re.sub(r"\s+", " ", s) for s in df.title]
df.publisher  = [re.sub(r"\s+", " ", s) for s in df.publisher]
df.authors = df.authors.str.replace('-', ' ')
df.publisher = df.publisher.str.replace('!', ' ')
df.title = df.title.str.replace('-', ' ')
df.title = df.title.str.replace('!', ' ')
df.title = df.title.str.replace('?', ' ')

# Display Dataframe rows
df.head()

In [ ]:
# Replace hypen in some dates to forward slash to make the representation uniform
df.publication_date = df.publication_date.str.replace('-', '/')

# Split & reconstitute each date to make the numeral representation uniform
df.publication_date =  [str(int(x.split('/')[0])) + '/' +  str(int(x.split('/')[1])) + '/' +  str(int(x.split('/')[2])) for x in df.publication_date]

# Conversion to date throws an error. So using Error handling to display the index of the errors & combine the indices in a list
err = []

for i, d in enumerate(df.publication_date):
    try:
        pd.to_datetime(d)
    except: 
        err.append(i)

# Print the rows with errors. Both dates show 31 days in June & November which is not possible. 
print('Date Error Rows :', '\n', df.loc[err,'publication_date'], '\n')

# Manually inputting dates in these rows via Goodreads Website
df.loc[8181, 'publication_date'] = '10/31/2000'
df.loc[11098, 'publication_date'] = '6/30/1982'

# Datetime conversion now runs error free
df.publication_date = pd.to_datetime(df.publication_date)

# Display all column datatypes
df.dtypes

In [ ]:
# Remove the parenthesized parts from authors & title columns; also create a new column 'raw title' to preserve old info
df.authors = df.authors.str.replace(r"\s*\(.*\)\s*", "", regex=True).str.lower()
df['raw_title'] = df.title.str.replace(r"\s*\(.*\)\s*", "", regex=True).str.lower()

# Removing accents from alphabets (for example, 'á' replaced with 'a')
df.title = df.title.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Removing unwanted whitespaces from columns
df.authors = df.authors.str.strip()
df.publisher = df.publisher.str.strip()
df.title = df.title.str.strip()
df.raw_title = df.raw_title.str.strip()

In [ ]:
# Removing Noise from publisher column & storing it in a new column to preserve old info

df['raw_pub'] = df.publisher.str.replace(r"\s*\(.*\)\s*", "", regex=True).str.lower()
df.raw_pub = df.raw_pub.str.strip()
df.raw_pub = df.raw_pub.str.replace(' ltd.', ' ltd').str.replace(' limited', ' ltd').str.replace(' ltd', '')
df.raw_pub = df.raw_pub.str.replace('&', 'and')
df.raw_pub = df.raw_pub.str.replace(' company', ' co')
df.raw_pub = df.raw_pub.str.replace(' co ', ' co.').str.replace(' co.', '').str.replace(' co', '')
df.raw_pub = df.raw_pub.str.replace(' inc.', ' inc').str.replace(' inc', '')
df.raw_pub = df.raw_pub.str.replace("'", '').str.replace("-", ' ')
df.raw_pub = df.raw_pub.str.replace(". ", '.').str.replace(".", '. ')
df.raw_pub = df.raw_pub.str.replace(' plc', '').str.replace(' llc', '')
df.raw_pub = df.raw_pub.str.replace('books', 'book').str.replace('book','')
df.raw_pub = df.raw_pub.str.replace('classics', 'classic').str.replace('classic','classics').str.replace('classics','')
df.raw_pub = df.raw_pub.str.replace('publishers', 'publisher').str.replace('publisher','')
df.raw_pub = df.raw_pub.str.replace('publishing group', 'publishing').str.replace('publishing', '').str.replace(' group', '')
df.raw_pub = df.raw_pub.str.replace('publications', 'publication').str.replace('publication','')
df.raw_pub = df.raw_pub.str.replace('productions', '').str.replace('paperbacks','paperback').str.replace('paperback','')
df.raw_pub = df.raw_pub.str.replace('editions', '').str.replace('enterprises','').str.replace(' press','').str.replace(' publ.','')
df.raw_pub = df.raw_pub.str.replace(' hardcover', '').str.replace('u. s. a', 'usa').str.replace('u. k', 'uk')
df.raw_pub = df.raw_pub.str.replace(' usa', '').str.replace(' uk', '')
df.raw_pub = df.raw_pub.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# dropping non-informative & erroneous column 'isbn13'
df.drop('isbn13', axis=1, inplace=True)

# Seperate books with multiple authors into a list of the different author names 
df['authors_comb'] = df.authors.str.split('/')

# Creating seperate columns for multiple authors
authors = df.authors.str.split('/', expand=True)
cols = ['author_'+ str(x+1) for x in authors.columns]
authors.columns = cols 

#Concatenating with the parent dataframe
df = pd.concat([df,authors], axis=1)
df.replace({None: np.nan}, inplace=True)

# Dropping Authors without ratings which won't be useful for the model
no_ratings = df[df.average_rating==0].index
df = df.drop(no_ratings,axis=0)

df.head()

In [ ]:
# Saving the cleaned file to the project directory
df.to_csv(r"C:\Users\Ashis\Desktop\ML Project\Project 1\books_cleaned.csv")